In [ ]:
#
# 本程式為示範不利用sklearn.kmeans開放源,改成自行寫kmeans程式
#
import random
import sys
import matplotlib.pyplot as plt #畫圖
# 畫圖
def draw(dx,classify,distortions,best,times):
    color = ['orange','c', 'lightgreen','lavender','pink', 'gray', 'purple', 'k', 'm', 'y', 'g', 'b']
    # 母畫布
    fig = plt.figure()
    # 誤差平方和SSE
    fig.add_subplot(221)
    plt.title('SSE (elbow method)',fontsize=10)
    plt.plot(range(times),distortions)
    plt.plot(best,distortions[best],"go")
    plt.xlabel("times")
    # 原始數據
    fig.add_subplot(223)
    plt.title('Original data',fontsize=10)
    for data in dx:
        plt.scatter(data[0],data[1],color='black')
    # 最好的分類結果
    fig.add_subplot(224)
    plt.title('result',fontsize=10)
    for i in range(len(dx)):
        plt.scatter(dx[i][0],dx[i][1],color=color[classify[i]])
    fig.tight_layout()
    plt.show()
def SSE(data,classify,center):
    distances= 0
    for pos in range(len(data)):
        cluster = classify[pos]
        for Dim in range((len(data[pos]))):
            distances += ((data[pos][Dim] - center[cluster][Dim])**2)
            
    return distances
def kMeans(k,data,center,classify,tmp,Dimension): # 只執行kMeans的一次iteration
    clusterPlus = []
    # 每一群的所有點座標值加總
    for c in range(k):
        clusterPlus.append([0 for Dim in range(Dimension)])
    # 每一群共有多少個點
    dotNum = [0 for c in range(k)]

    for pos in range(len(data)):   # 將每個點分到最近的群心
        mini = sys.float_info.max # 預設mini為最大數字
        for c in range(k): # 群心
            dis = 0
            # 計算點到群心的距離
            for Dim in range(Dimension): # 維度
                dis += ((data[pos][Dim] - center[c][Dim])**2)
            # 距離該群心比較近
            if mini > dis:
                mini = dis
                close_center = c
        classify.append(close_center)  # 將該點所分配的群編號加入classify
        
        for Dim in range(Dimension):
            clusterPlus[close_center][Dim] += data[pos][Dim] # 將該點座標值加到對應的群心座標值
        dotNum[close_center] += 1  # 紀錄該群的點數，以利計算群心平均座標值

    # 預防某一群沒有任何點
    if 0 in dotNum:
        return False
    # 分完後重新計算群心
    for c in range(k):
        for Dim in range(Dimension):
            center[c][Dim] = clusterPlus[c][Dim]/dotNum[c]
    # 群心不變動->收斂 (位置可能不一樣)
    n = 0
    for c in range(k):
        if center[c] in tmp: #tmp為前一輪的所有群心
            n += 1
        if n == k: # 所有群心都沒變動->收斂
            return True
    return False

def result(k,data):
    center = []
    Dimension = len(data[0])
    classify = []
    # 隨機群心
    for c in range(k):
        contain = []
        for Dim in range(Dimension):
            r = round(random.uniform(min(data)[Dim], max(data)[Dim]),1)
            contain.append(r)
        center.append(contain)
    # 最大收斂次數
    for i in range(200): # 設定k-means最多iteration(迭代)次數
        tmp = []
        classify.clear()
        # 複製群心位置
        tmp = center.copy()
        # 分類
        if kMeans(k,data,center,classify,tmp,Dimension) == True:
            return classify,center # k-means若已經收斂,就不需做到最多iteration(迭代)次數
    return classify,center # 若k-means做到最多iteration(迭代)次數都尚未收斂,則傳回目前分群結果
def main():
    data = [[-39,32],[-22,38],[-40,23],[-33,37],[-28,37],[-5,-1],[12,1],[9,-11],[26,31],[28,25],[35,30],[37,23],[29,29],[0,0]]
    k = int(input("要分幾群? "))
    times = 10  # k-means 獨立實驗次數
    score = []  # SSE
    results = [] # 分類結果
    centers = [] # 中心點
    for i in range(times):
        
        classify,center = result(k,data) # 做完整k-means分群結果
        results.append(classify)
        centers.append(center)
        score.append(SSE(data,classify,center)) # 計算分數
    # 最好的結果
    best = score.index(min(score))
    # 畫圖
    draw(data,results[best],score,best,times)
if __name__ == '__main__':
    main()